<a href="https://colab.research.google.com/github/TheresiaQuintes/ml/blob/main/_downloads/76d764ad694d0795e494a1edbfb068a6/buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on a hardware accelerator like the
GPU or MPS, if available. Let\'s check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are
available, otherwise we use the CPU.


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [10]:
X = torch.rand(2, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9, 8])


In [12]:
print(logits)
print(logits.shape)
print(pred_probab)

tensor([[-0.0685,  0.0251, -0.0947, -0.0583, -0.0427,  0.0165,  0.0058,  0.1247,
          0.0710,  0.1496],
        [-0.0387,  0.0030, -0.0960, -0.0687, -0.0489, -0.0008,  0.0233,  0.0641,
          0.0867,  0.0502]], grad_fn=<AddmmBackward0>)
torch.Size([2, 10])
tensor([[0.0919, 0.1009, 0.0895, 0.0928, 0.0943, 0.1001, 0.0990, 0.1115, 0.1057,
         0.1143],
        [0.0963, 0.1004, 0.0909, 0.0934, 0.0953, 0.1000, 0.1025, 0.1067, 0.1092,
         0.1053]], grad_fn=<SoftmaxBackward0>)


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [13]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [14]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [17]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1158, -0.0393,  0.7311,  0.0304, -0.4957,  0.2362, -0.5703, -0.2676,
          0.1793, -0.5238,  0.2306, -0.0517, -0.4693,  0.1206,  0.2932, -0.2812,
         -0.4031,  0.2293, -0.6183,  0.6636],
        [ 0.1567,  0.5341,  0.6296,  0.0843, -0.6544, -0.2004, -0.7406, -0.4111,
          0.0172, -0.4804, -0.0177, -0.3103, -0.1503,  0.1687,  0.1298, -0.1841,
         -0.1287,  0.2168, -0.3309,  0.6146],
        [ 0.3450,  0.5454,  0.4025,  0.3941, -0.4671, -0.0454, -0.6965, -0.4040,
          0.0700, -0.2801,  0.2408, -0.1693, -0.0897,  0.3951,  0.1377, -0.2144,
         -0.3534,  0.4399, -0.3596,  0.4657]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1158, 0.0000, 0.7311, 0.0304, 0.0000, 0.2362, 0.0000, 0.0000, 0.1793,
         0.0000, 0.2306, 0.0000, 0.0000, 0.1206, 0.2932, 0.0000, 0.0000, 0.2293,
         0.0000, 0.6636],
        [0.1567, 0.5341, 0.6296, 0.0843, 0.0000, 0.0000, 0.0000, 0.0000, 0.0172,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1687, 0.12

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [18]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [19]:
print(logits)

tensor([[ 0.1392,  0.4623, -0.0675, -0.3462, -0.0470, -0.1672, -0.2351,  0.1666,
          0.1567, -0.2157],
        [ 0.1859,  0.3913, -0.2022, -0.3378,  0.0291, -0.1108, -0.1537,  0.2653,
          0.1818, -0.2770],
        [ 0.1500,  0.3798, -0.0366, -0.2730, -0.0960, -0.1222, -0.1013,  0.2140,
          0.1218, -0.1333]], grad_fn=<AddmmBackward0>)


nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [20]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [24]:
model.named_parameters()

<generator object Module.named_parameters at 0x7b06f1218660>

In [28]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : Parameter containing:
tensor([[ 0.0240, -0.0147,  0.0124,  ...,  0.0134, -0.0323,  0.0161],
        [-0.0225,  0.0192,  0.0275,  ..., -0.0227, -0.0308, -0.0272],
        [-0.0356, -0.0136,  0.0204,  ..., -0.0044,  0.0222, -0.0111],
        ...,
        [-0.0012, -0.0201, -0.0273,  ..., -0.0162,  0.0258,  0.0061],
        [-0.0173, -0.0043, -0.0229,  ..., -0.0315, -0.0266, -0.0029],
        [ 0.0241, -0.0294, -0.0212,  ...,  0.0336,  0.0313, -0.0044]],
       requires_grad=True) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : Parameter containing:
tensor([-2

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
